Logic of web-scraping
We begin by identifying a web page containing information we are interested in collecting. Then we need to know the following:

The location (i.e., web address) where the web page can be accessed. For example, the UK Data Service homepage can be accessed via https://ukdataservice.ac.uk/.
The location of the information we are interested in within the structure of the web page. This involves visually inspecting a web page's underlying code using a web browser.
And do the following:

Request the web page using its web address.
Parse the structure of the web page so your programming language can work with its contents.
Extract the information we are interested in.
Write this information to a file for future use.
For any programming task, it is useful to write out the steps needed to solve the problem: we call this pseudo-code, as it is captures the main tasks and the order in which they need to be executed.

For our first example, let's convert the steps above into executable Python code for capturing data about Covid-19.

Example: Capturing Covid-19 data
Worldometer is a website that provides up-to-date statistics on the following domains: the global population; food, water and energy consumption; environmental degradation and many others (known as its Real Time Statistics Project). In its own words:[1]

Worldometer is run by an international team of developers, researchers, and volunteers with the goal of making world statistics available in a thought-provoking and time relevant format to a wide audience around the world. Worldometer is owned by Dadax, an independent company. We have no political, governmental, or corporate affiliation.

Since the outbreak of Covid-19 it has provided regular daily snapshots on the progress of this disease, both globally and at a country level.


Identifying the web address
The website can be accessed here: https://www.worldometers.info/coronavirus/

Let's work through the steps necessary to collect data about the number of Covid-19 cases, deaths and recoveries globally.

First, let's become familiar with this website: click on the link below to view the web page in your browser: https://www.worldometers.info/coronavirus/

(Note: it possible to load websites into Python in order to view them, however the website we are interested in doesn't allow this. See the example code below for how it would work for a different website - just remove the quotation marks enclosing the code and run the cell).


In [1]:
"""
from IPython.display import IFrame

IFrame("https://ukdataservice.ac.uk/", width="600", height="650")
"""


'\nfrom IPython.display import IFrame\n\nIFrame("https://ukdataservice.ac.uk/", width="600", height="650")\n'

Locating information
The statistics we need are near the top of the page under the following headings:

Coronavirus Cases:
Deaths:
Recovered:
However, we need more information than this in order to scrape the statistics. Websites are written in a langauge called HyperText Markup Language (HTML), which can be understood as follows:[2]

HTML describes the structure of a web page
HTML consists of a series of elements
HTML elements tell the browser how to display the content
HTML elements are represented by tags
HTML tags label pieces of content such as "heading", "paragraph", "table", and so on
Browsers do not display the HTML tags, but use them to render the content of the page
Visually inspecting the underlying HTML code
Therefore, what we need are the tags that identify the section of the web page where the statistics are stored. We can discover the tags by examining the source code (HTML) of the web page. This can be done using your web browser: for example, if you use use Firefox you can right-click on the web page and select View Page Source from the list of options.

TASK: Try this yourself with the Worldometer web page.

The snippet below shows sample source code for the section of the Covid-19 web page we are interested in.


In the above example, we can see multiple tags that contain various elements (e.g., text content, other tags). For instance, we can see that the Covid-19 statistics are enclosed in <span><\span> tags, which themselves are located within <div><\div> tags.

As you can see, exploring and locating the contents of a web page remains a manual and visual process, and in Brooker's estimation (2020, 252):

Hence, more so than the actual Python, it's the detective work of unpicking the internal structure of a webpage that is probably the most vital skill here.

Requesting the web page
Now that we possess the necessary information, let's begin the process of scraping the web page. There is a preliminary step, which is setting up Python with the modules it needs to perform the web-scrape.


In [4]:
!pip install bs4

Using legacy 'setup.py install' for bs4, since package 'wheel' is not installed.
    Running setup.py install for bs4: started
    Running setup.py install for bs4: finished with status 'done'


In [5]:
# Import modules

import os # module for navigating your machine (e.g., file directories)
import requests # module for requesting urls
import csv # module for handling csv files
import pandas as pd # module for handling data
from datetime import datetime # module for working with dates and time
from bs4 import BeautifulSoup as soup # module for parsing web pages
print("Succesfully imported necessary modules")


Succesfully imported necessary modules


Modules are additional techniques or functions that are not present when you launch Python. Some do not even come with Python when you download it and must be installed on your machine separately - think of using ssc install <package> in Stata, or install.packages(<package>) in R. For now just understand that many useful modules need to be imported every time you start a new Python session.

Now, let's implement the process of scraping the page. First, we need to request the web page using Python; this is analogous to opening a web browser and entering the web address manually. We refer to a page's location on the internet as its web address or Uniform Resource Locator (URL).

In [6]:
# Define the URL where the web page can be accessed

url = "https://www.worldometers.info/coronavirus/"

# Request the web page from the URL

response = requests.get(url, allow_redirects=True) # request the url
# response.headers
response.status_code # check if page was requested successfully

200

First, we define a variable (also known as an 'object' in Python) called url that contains the web address of the page we want to request. Next, we use the get() method of the requests module to request the web page, and in the same line of code, we store the results of the request in a variable called response. Finally, we check whether the request was successful by calling on the status_code attribute of the response variable.

Confused? Don't worry, the conventions of Python and using its modules take a bit of getting used to. At this point, just understand that you can store the results of commands in variables, and a variable can have different attributes that can be accessed when needed. Also note that you have a lot of freedom in how you name your variables (subject to certain restrictions - see here for some guidance).

For example, the following would also work:

web_address = "https://www.worldometers.info/coronavirus/"

scrape_result = requests.get(web_address, allow_redirects=True)
scrape_result.status_code
Back to the request:

Good, we get a status code of 200 - this means we successfully requested the web page. Lau, Gonzalez and Nolan provide a succinct description of different types of response status codes:

100s - Informational: More input is expected from client or server (e.g. 100 Continue, 102 Processing)
200s - Success: The client's request was successful (e.g. 200 OK, 202 Accepted)
300s - Redirection: Requested URL is located elsewhere; May need user's further action (e.g. 300 Multiple Choices, 301 Moved Permanently)
400s - Client Error: Client-side error (e.g. 400 Bad Request, 403 Forbidden, 404 Not Found)
500s - Server Error: Server-side error or server is incapable of performing the request (e.g. 500 Internal Server Error, 503 Service Unavailable)
For clarity:

Client: your machine
Server: the machine you are requesting the web page from
You may be wondering exactly what it is we requested: if you were to type the URL (https://www.worldometers.info/coronavirus/) into your browser and hit enter, the web page should appear on your screen. This is not the case when we request the URL through Python but rest assured, we have successfully requested the web page. To see the content of our request, we can examine the text attribute of the response variable:

In [9]:
response.text[:1000]


'\n<!DOCTYPE html>\n<!--[if IE 8]> <html lang="en" class="ie8"> <![endif]-->\n<!--[if IE 9]> <html lang="en" class="ie9"> <![endif]-->\n<!--[if !IE]><!-->\n<html lang="en">\n<!--<![endif]-->\n<head>\n<meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<title>COVID Live - Coronavirus Statistics - Worldometer</title>\n<meta name="description" content="Live statistics and coronavirus news tracking the number of confirmed cases, recovered patients, tests, and death toll due to the COVID-19 coronavirus from Wuhan, China. Coronavirus counter with new cases, deaths, and number of tests per 1 Million population. Historical data and info. Daily charts, graphs, news and updates">\n\n<link rel="shortcut icon" href="/favicon/favicon.ico" type="image/x-icon">\n<link rel="apple-touch-icon" sizes="57x57" href="/favicon/apple-icon-57x57.png">\n<link rel="apple-touch-icon" sizes="60x60" href="/favicon/apple

This shows us a sample of the underlying code (HTML) of the web page we requested. It should be obvious that in its current form, the result of this request will be difficult to work with. This is where the BeautifulSoup module comes in handy.

(See Appendix A for more examples of how the requests module works and what information it returns.)

Parsing the web page

Now it's time to identify and understand the structure of the web page we requested. We do this by converting the content contained in the response.text attribute into a BeautifulSoup variable. BeautifulSoup is a Python module that provides a systematic way of navigating the elements of a web page and extracting its contents. Let's see how it works in practice:



In [8]:
# Extract the contents of the webpage from the response

soup_response = soup(response.text, "html.parser") # Parse the text as a Beautiful Soup object
soup_sample = soup(response.text[:1000], "html.parser") # Parse a sample of the text
soup_sample



<!DOCTYPE html>

<!--[if IE 8]> <html lang="en" class="ie8"> <![endif]-->
<!--[if IE 9]> <html lang="en" class="ie9"> <![endif]-->
<!--[if !IE]><!-->
<html lang="en">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<title>COVID Live - Coronavirus Statistics - Worldometer</title>
<meta content="Live statistics and coronavirus news tracking the number of confirmed cases, recovered patients, tests, and death toll due to the COVID-19 coronavirus from Wuhan, China. Coronavirus counter with new cases, deaths, and number of tests per 1 Million population. Historical data and info. Daily charts, graphs, news and updates" name="description"/>
<link href="/favicon/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<link href="/favicon/apple-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
<link href="/favicon/apple-icon-60x60.png" rel="apple-touch-icon" sizes="6

BeautifulSoup has taken the unstructured text contained in response.text and parsed it as HTML: now we can clearly see the hierarchical structure and tags that comprise a web page's HTML.

Note again how we call on a method (soup()) from a module (BeautifulSoup) and store the results in a variable (soup_response).

Of course, we've only displayed a sample of the code here for readability. What about the full text contained in soup_response: how do we navigate such voluminous results? Thankfully the BeautifulSoup module provides some intuitive methods for doing so.

TASK: view the full contents of the web page stored in the variable soup_response.


In [10]:
# INSERT CODE HERE

Extracting information
Now that we have parsed the web page, we can use Python to navigate and extract the information of interest. To begin with, let's locate the section of the web page containing the overall Covid-19 statistics.



In [12]:
# Find the sections containing the data of interest

sections = soup_response.find_all("div", id="maincounter-wrap")
sections

[<div id="maincounter-wrap" style="margin-top:15px">
 <h1>Coronavirus Cases:</h1>
 <div class="maincounter-number">
 <span style="color:#aaa">547,512,638 </span>
 </div>
 </div>,
 <div id="maincounter-wrap" style="margin-top:15px">
 <h1>Deaths:</h1>
 <div class="maincounter-number">
 <span>6,347,923</span>
 </div>
 </div>,
 <div id="maincounter-wrap" style="margin-top:15px;">
 <h1>Recovered:</h1>
 <div class="maincounter-number" style="color:#8ACA2B ">
 <span>522,845,791</span>
 </div>
 </div>]

We used the find_all() method to search for all <div> tags where the id="maincounter-wrap". And because there is more than one set of tags matching this id, we get a list of results. We can check how many tags match this id by calling on the len() function:

In [13]:
len(sections)

3

We can view each element in the list of results as follows:

In [14]:
for section in sections:
    print("--------")
    print(section)
    print("--------")
    print("\r") # print some blank space for better formatting

--------
<div id="maincounter-wrap" style="margin-top:15px">
<h1>Coronavirus Cases:</h1>
<div class="maincounter-number">
<span style="color:#aaa">547,512,638 </span>
</div>
</div>
--------

--------
<div id="maincounter-wrap" style="margin-top:15px">
<h1>Deaths:</h1>
<div class="maincounter-number">
<span>6,347,923</span>
</div>
</div>
--------

--------
<div id="maincounter-wrap" style="margin-top:15px;">
<h1>Recovered:</h1>
<div class="maincounter-number" style="color:#8ACA2B ">
<span>522,845,791</span>
</div>
</div>
--------



We are nearing the end of our scrape. The penultimate task is to extract the statistics within the <span> tags and store them in some variables. We do this by accessing each item in the sections list using its positional value (index).

In [15]:
cases = sections[0].find("span").text.replace(" ", "").replace(",", "")
deaths = sections[1].find("span").text.replace(",", "")
recov = sections[2].find("span").text.replace(",", "")
print("Number of cases: {}; deaths: {}; and recoveries: {}.".format(cases, deaths, recov))


Number of cases: 547512638; deaths: 6347923; and recoveries: 522845791.


The above code performs a couple of operations:

For each item (i.e., set of <div> tags) in the list, it finds the <span> tags and extracts the text enclosed within them.
We clean the text by removing blank spaces and commas.
In this example, referring to an item's positional index works because our list of <div> tags stored in the sections variable is ordered: the tag containing the number of cases appears before the tag containing the number of deaths, which appears before the tag containing the number of recovered patients.

In Python, indexing begins at zero (in R indexing begins at 1). Therefore, the first item in the list is accessed using sections[0], the second using sections[1] etc.

(To learn more about lists in Python, see Chapter 22 of *How to Code in Python 3*)

Saving results from the scrape
The final task is to save the variables to a file that we can use in the future. We'll write to a Comma-Separated Values (CSV) file for this purpose, as it is an open-source, text-based file format that is commonly used for sharing data on the web.


In [17]:
# Create a downloads folder

try:
    os.mkdir("./downloads")
except:
    print("Unable to create folder: already exists")

The use of "./" tells the os.mkdir() command that the "downloads" folder should be created at the same level of the directory where this notebook is located. So if this notebook was stored in a directory located at "C:/Users/joebloggs/notebooks", the os.mkdir() command would result in a new folder located at "C:/Users/joebloggs/notebooks/downloads".

(Technically the "./" is not needed and you could just write os.mkdir("downloads") but it's good practice to be explicit)


In [18]:
# Write the results to a CSV file

date = datetime.now().strftime("%Y-%m-%d") # get today's date in YYYY-MM-DD format
print(date)

variables = ["Cases", "Deaths", "Recoveries"] # define variable names for the file
outfile = "./downloads/covid-19-statistics-" + date + ".csv" # define a file for writing the results
obs = cases, deaths, recov # define an observation (row)

with open(outfile, "w", newline="") as f: # with the file open in "write" mode, and giving it a shorter name (f)
    writer = csv.writer(f) # define a 'writer' object that allows us to export information to a CSV
    writer.writerow(variables) # write the variable names to the first row of the file
    writer.writerow(obs) # write the observation to the next row in the file


2022-06-24


The code above defines some headers and a name and location for the file which will store the results of the scrape. We then open the file in write mode, and write the headers to the first row, and the statistics to subsequent rows.

How do we know this worked? The simplest way is to check whether a) the file was created, and b) the results were written to it.

In [19]:
# Check presence of file in "downloads" folder

os.listdir("./downloads")
# Open file and read (import) its contents

with open(outfile, "r") as f:
    data = f.read()
    
print(data)    


Cases,Deaths,Recoveries
547512638,6347923,522845791



In [ ]:
# https://www.geeksforgeeks.org/downloading-pdfs-with-python-using-requests-and-beautifulsoup/

# Import libraries
import requests
from bs4 import BeautifulSoup
  
# URL from which pdfs to be downloaded
url = "https://www.geeksforgeeks.org/how-to-extract-pdf-tables-in-python/"
  
# Requests URL and get response object
response = requests.get(url)
  
# Parse text obtained
soup = BeautifulSoup(response.text, 'html.parser')
  
# Find all hyperlinks present on webpage
links = soup.find_all('a')
  
i = 0
  
# From all links check for pdf link and
# if present download file
for link in links:
    if ('.pdf' in link.get('href', [])):
        i += 1
        print("Downloading file: ", i)
  
        # Get response object for link
        response = requests.get(link.get('href'))
  
        # Write content in pdf file
        pdf = open("pdf"+str(i)+".pdf", 'wb')
        pdf.write(response.content)
        pdf.close()
        print("File ", i, " downloaded")
  
print("All PDF files downloaded")